In [1]:
!pip install netCDF4
!pip install basemap

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import glob
from datetime import datetime, date, timedelta 
from pathlib import Path
from mpl_toolkits.basemap import Basemap

In [ ]:
files = glob.glob(r'data/2021/*/*.nc')

for f in files:
    nc = Dataset(f)
    plt.close()
    lon = np.transpose(nc.variables['REFERENCE_POINT_LON'][:])
    lat = np.transpose(nc.variables['REFERENCE_POINT_LAT'][:])
    radiance = nc.variables['RADIANCE'][:]
    wavelength = nc.variables['WAVELENGTH'][:]
    wave_len = len(wavelength) / 2
    wave_len = int(wave_len)
    wave_len2 = len(wavelength[0])/ 2
    wave_len2 = int(wave_len2)
    sza = np.transpose(nc.variables['SOLAR_ZENITH_ANGLE'][:])
    O5s_ids = np.argwhere((135.0 <= wavelength[wave_len,wave_len2,:]) & (wavelength[wave_len,wave_len2,:] <= 137.0))
    O5s = np.nansum(radiance[:,:,O5s_ids],axis = 2)*.04
    O5s = np.transpose(O5s[:,:,0])
        
    LBH_ids = np.argwhere((141.0 < wavelength[wave_len,wave_len2,:]) & (wavelength[wave_len,wave_len2,:] < 153.0))
    LBHs = np.nansum(radiance[:,:,LBH_ids],axis = 2)*.04
    LBHs = np.transpose(LBHs[:,:,0])

    mp = Basemap(projection='geos',
            rsphere=(6378137.00,6356752.3142),
            resolution='c',
            lon_0=-45)

    x,y = mp(lon,lat)
    hour = f[40:42]
    minutes = f[43:45]
    day_num = f[36:39]
    day_num.rjust(13 + len(day_num), '0') 
    year = 2021
    start_date = date(int(year), 1, 1)
    result_date = start_date + timedelta(days=int(day_num) - 1) 
    result = result_date.strftime("%Y-%m-%d")
    Path("plots/" + result).mkdir(parents=True, exist_ok=True)
    Path("plots/" + result + "/1356").mkdir(parents=True, exist_ok=True)
    Path("plots/" + result + "/LBH").mkdir(parents=True, exist_ok=True)
    Path("plots/" + result + "/SZA").mkdir(parents=True, exist_ok=True)
    Path("plots/" + result + "/plot").mkdir(parents=True, exist_ok=True)
    if 'CHA' in f:
        channel = "CHA"
    
    if 'CHB' in f:
        channel = "CHB"
    
    c_scheme = mp.pcolor(x,y,O5s,cmap = 'jet',vmin=0,vmax=40)
    plt.colorbar(c_scheme)
    if 'CHA' in f: 
        plt.title("Type: L1C CHA NI1 1356 \n Date: "+ result + " " + hour + ":" + minutes)
    
    if 'CHB' in f: 
        plt.title("Type: L1C CHB NI1 1356 \n Date: "+ result + " " + hour + ":" + minutes)
    mp.drawcoastlines()
   

    plt.savefig("plots/" + result + "/1356/" + result + "_" + hour + "_" + minutes + "_" + channel +"_1356.png")
    
    plt.close()
    c_scheme = mp.pcolor(x,y,LBHs,cmap = 'jet',vmin=0,vmax=80)
    plt.colorbar(c_scheme)
    if 'CHA' in f:
        plt.title("Type: L1C CHA NI1 LBH \n Date: "+ result + " " + hour + ":" + minutes)
        
    if 'CHB' in f:
        plt.title("Type: L1C CHB NI1 LBH \n Date: "+ result + " " + hour + ":" + minutes)
    mp.drawcoastlines()
 

    plt.savefig("plots/" + result + "/LBH/" + hour + "_" + minutes + "_" + channel +"_LBH.png")
    
    plt.close()
    c_scheme = mp.pcolor(x,y,sza,cmap = 'RdGy',vmin=0,vmax=180)
    plt.colorbar(c_scheme)
    if 'CHA' in f:
        plt.title("Type: L1C CHA NI1 SZA \n Date: "+ result + " " + hour + ":" + minutes)
        
    if 'CHB' in f:
        plt.title("Type: L1C CHB NI1 SZA \n Date: "+ result + " " + hour + ":" + minutes)
    mp.drawcoastlines()


    plt.savefig("plots/" + result + "/SZA/" + hour + "_" + minutes + "_" + channel +"_SZA.png")

    plt.close()
    c_scheme = plt.pcolor(lon,lat,O5s,cmap = 'jet',vmin=0,vmax=40)
    plt.colorbar(c_scheme)
    if 'CHA' in f:
        plt.title("Type: L1C CHA NI1 1356 \n Date: "+ result + " " + hour + ":" + minutes)
        
    if 'CHB' in f:
        plt.title("Type: L1C CHB NI1 1356 \n Date: "+ result + " " + hour + ":" + minutes)
    plt.savefig("plots/" + result + "/plot/" + hour + "_" + minutes + "_" + channel +"_1356_plot.png")


/opt/conda/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:3395: RuntimeWarning: invalid value encountered in greater
  x = ma.masked_values(np.where(x > 1.e20,1.e20,x), 1.e20)
/opt/conda/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:3396: RuntimeWarning: invalid value encountered in greater
  y = ma.masked_values(np.where(y > 1.e20,1.e20,y), 1.e20)
<ipython-input-5-41d4e8cfb052>:61: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  c_scheme = mp.pcolor(x,y,O5s,cmap = 'jet',vmin=0,vmax=40)
<ipython-input-5-41d4e8cfb052>:74: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, 